##### Import all libraries we will need for this ETL:

In [ ]:
from instagram.client import InstagramAPI
import requests
import json
import pandas as pd
import time
import datetime as datetime
import csv
import pprint

##### Server side flow to get access token:

In [19]:
# Authorization URL to get access token: 'https://api.instagram.com/oauth/authorize/?client_id=[ENTER CLIEND ID HERE]&redirect_uri=[ENTER REDIRECT-URI HERE]&response_type=token'

##### Insert your access token and client secret credentials:

In [3]:
access_token = ''
client_secret = ''

##### We will scrape the Media API endpoint:

In [4]:
recentMediaResponse = requests.get("https://api.instagram.com/v1/users/self/media/recent/",params = {"access_token": access_token})
recentMediaJson = json.loads(recentMediaResponse.text)
pprint.pprint(recentMediaJson)

{u'data': [{u'attribution': None,
            u'caption': {u'created_time': u'1505099111',
                         u'from': {u'full_name': u'',
                                   u'id': u'25964371',
                                   u'profile_picture': u'https://scontent.cdninstagram.com/t51.2885-19/s150x150/13741198_302807110065086_217692851_a.jpg',
                                   u'username': u'izzy_1990'},
                         u'id': u'17898339034006102',
                         u'text': u'\u270c\ufe0f\U0001f60e'},
            u'comments': {u'count': 0},
            u'created_time': u'1505099111',
            u'filter': u'Clarendon',
            u'id': u'1601209858639478027_25964371',
            u'images': {u'low_resolution': {u'height': 296,
                                            u'url': u'https://scontent.cdninstagram.com/t51.2885-15/s320x320/e35/21479622_1946480525619573_6980610981843959808_n.jpg',
                                            u'width': 320},
      

##### Here we begin transforming the JSON file into a pandas dataframe and time stamp to load into a MySQL database: 

In [17]:
Created_Time,Caption,Type,Filter,Link,Comments,Likes,Tags,Id = [],[],[],[],[],[],[],[],[]
for result in recentMediaJson['data']:
        
    Created_Time.append(result['created_time'])
    Caption.append(result['caption']['text']),
    Type.append(result['type']),
    Filter.append(result['filter']),
    Link.append(result['link']),
    Comments.append(result['comments']['count']),
    Likes.append(result['likes']['count']),
    Tags.append(result['tags']),
    Id.append(result['id'])
        
df = pd.DataFrame([Created_Time,Caption,Type,Filter,Link,Comments,Likes,Tags,Id]).T

st = time.time()
st = datetime.datetime.fromtimestamp(st).strftime('%Y-%m-%d %H:%M:%S')
df['PullTime'] = st

DF = df.rename(columns={0L:'Created_Time', 1L:'Caption',2L:'Type', 3L:'Filter', 4L:'Link', 5L:'Comments', 6L:'Likes', 7L:'Tags', 8L:'Id' })

print DF

   Created_Time                                            Caption   Type  \
0    1505099111                                               ✌️😎  image   
1    1505096915                                    😎✌️❤️ #poolday  image   
2    1504069239  Mood #poodle #dogsofinstagram #minipoodle #sle...  image   
3    1504068976                 I ❤️ these queens 👸🏾 to death 💀  image   
4    1503636445  The best part of the song has to be the breakd...  image   
5    1503623394                    Popping champagne bottles 🥂 🍾  video   
6    1503623212                                              🎊 🦄  image   
7    1503430196  Data Scientists have nothing on me, watch the ...  video   
8    1503364831  Yaaassss 👠 feeling kweenty & ready to step on...  image   
9    1503351662                 Time to guzzle! #SinCity #GayVegas  image   
10   1503336250                                                 👀  image   
11   1503292907                                    View + Open Bar  video   
12   15032

##### We open a connection to our database and load the data frame into a table:

In [ ]:
db = create_engine('mysql+mysqlconnector://[user]:[pass]@[host]:[port]/[schema]', echo=False)
cnx = db.connect()
DF.to_sql(name = 'data_base_table_here', con= cnx, if_exists='append', index=True)
cnx.close()
db.dispose()

# This script is now ready to be scheduled via an automation scheduler like Jenkins or Windows Task Scheduler

Below is just for fun, writing to a CSV for a quick analysis of your data:

In [ ]:
filename = 'people.csv'
DF.to_csv(filename, index=False, encoding='utf-8')